# Sales Forecast - Model Training

Training AutoGluon TimeSeriesPredictor on the Store Sales dataset.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt

from src.data_loader import load_train_data, load_test_data
from src.forecaster import (
    prepare_timeseries_data,
    train_forecaster,
    load_forecaster,
    make_predictions
)

## Load and Prepare Data

In [ ]:
train_df = load_train_data()
print(f"Training data shape: {train_df.shape}")
train_df.head()

In [ ]:
# Convert to AutoGluon TimeSeriesDataFrame
ts_data = prepare_timeseries_data(train_df)
print(f"Number of unique time series: {len(ts_data.item_ids)}")
ts_data.head()

## Train Model

Training with `medium_quality` preset. Adjust `time_limit` based on available time.

In [ ]:
predictor = train_forecaster(
    train_data=ts_data,
    prediction_length=16,  # Forecast 16 days ahead
    time_limit=600,        # 10 minutes
    presets="medium_quality"
)

## Model Leaderboard

In [ ]:
predictor.leaderboard()

## Generate Predictions

In [ ]:
predictions = make_predictions(predictor, ts_data)
predictions.head(20)

## Visualize Predictions

In [ ]:
# Select a specific time series to visualize
sample_item = ts_data.item_ids[0]
print(f"Visualizing predictions for: {sample_item}")

# Get historical data for this item
historical = ts_data.loc[sample_item].tail(60)  # Last 60 days
forecast = predictions.loc[sample_item]

plt.figure(figsize=(14, 5))
plt.plot(historical.index, historical['target'], label='Historical', color='blue')
plt.plot(forecast.index, forecast['mean'], label='Forecast', color='red')
plt.fill_between(
    forecast.index,
    forecast['0.1'],
    forecast['0.9'],
    alpha=0.2,
    color='red',
    label='80% CI'
)
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title(f'Sales Forecast for {sample_item}')
plt.legend()
plt.show()

## Load Saved Model

For future use, load the trained model without retraining.

In [ ]:
# Load previously trained model
# predictor = load_forecaster()